## The compatibility of sQUlearn and sklearn

This notebook demonstrates the ability of sQUlearn to smoothly interact with sklearn routines. We first define a simple toy function and use the Chebychev quantum feature map to define a fidelity quantum kernel. The trainable parameters of the quantum feature map are sampled randomly. The aim in mind is to solve a regression problem using QSVR. Subsequently, we apply sklearn's cross validation score. Finally we demonstrate how to optimize the inherent hyperparameters of the SVR pipeline ($C$ and $\epsilon$) using GridSearchCV.

In [9]:
import numpy as np

from squlearn.feature_map import ChebPQC
from squlearn.kernel.matrix import FidelityKernel
from squlearn.kernel.ml import QSVR
from squlearn.util import Executor

In [10]:
# Generate data set (analogously to qgpr_workflow.ipynb example)
def func(x):
    return np.sin(3*x)*x*x

# define some noise that can be added to training data
noise_std = 0.1
x = np.linspace(-1, 1, 100)
y = func(x) 
x_train = np.linspace(-1, 1, 30)
y_train = func(x_train) + np.random.normal(scale=noise_std, size=len(x_train))

In [11]:
# We can define an qml method of choice and use additional sklearn functions
# to validate the model

# 1. Step: define QML method
# set up feature map with trainable parameters being randomly initialized
fmap = ChebPQC(
    num_qubits=4, num_features=1, num_layers=2
)
q_kernel_fqk = FidelityKernel(feature_map=fmap, executor=Executor('statevector_simulator'))
np.random.seed(0)
param = np.random.rand(fmap.num_parameters)
q_kernel_fqk.assign_parameters(param)
qsvr_fqk = QSVR(quantum_kernel=q_kernel_fqk, C=1e-3)

In [12]:
# 2. Step load and use additional sklearn routines
from sklearn.model_selection import cross_val_score, cross_val_predict

print("Cross validation score:", cross_val_score(qsvr_fqk, x_train.reshape(-1, 1), y_train, cv=5))
print("Cross validation prediction:", cross_val_predict(qsvr_fqk, x_train.reshape(-1, 1), y_train, cv=5))

Cross validation score: [-9.54521665 -1.32877969 -0.58454985 -3.03531856 -7.13851468]
Cross validation prediction: [ 0.07782236  0.07652342  0.0759526   0.07556558  0.07531007  0.07516507
  0.04408123  0.04448225  0.04494652  0.04546522  0.0460304   0.04663462
  0.02532761  0.02618817  0.02706792  0.027957    0.02884549  0.02972326
 -0.07687756 -0.07623228 -0.07561744 -0.0750417  -0.07451457 -0.07404682
 -0.08683296 -0.0870371  -0.08738593 -0.08790925 -0.08867848 -0.09041596]


In [13]:
# we could also use sklearn routines to optimize hyperparameters
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10], 'epsilon': [0.01, 0.1, 0.5]}
qsvr_grid = GridSearchCV(qsvr_fqk, param_grid)

In [14]:
qsvr_grid.fit(x_train.reshape(-1, 1), y_train)

GridSearchCV(estimator=QSVR(C=0.001, cache_size=200, epsilon=0.1, max_iter=-1,
                            quantum_kernel=<squlearn.kernel.matrix.fidelity_kernel.FidelityKernel object at 0x000001EA43790490>,
                            shrinking=True, tol=0.001, verbose=False),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'epsilon': [0.01, 0.1, 0.5]})

In [15]:
from sklearn.metrics import mean_squared_error as mse
print("Best parameters:", qsvr_grid.best_params_)
print("MSE with optimized hyperparameters:", mse(y_train, qsvr_grid.predict(x_train.reshape(-1, 1))))

Best parameters: {'C': 1, 'epsilon': 0.01}
MSE with optimized hyperparameters: 0.016102807832371526
